In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim import corpora
from gensim.models import LdaModel
import pandas as pd
import datetime

## 主题分析

In [2]:
df = pd.read_excel('data分词后.xlsx', header=0)
all_texts = [] # 创建一个空列表来存储句子
all_words = [] # 创建一个空列表来存储词语
for i in range(len(df)):
    content = df.分词后内容[i]
    all_texts.append(content)
    all_words.append(content.split(' '))

In [3]:
print('开始时间：' + str(datetime.datetime.now()))
dictionary = corpora.Dictionary(all_words)
corpus = [dictionary.doc2bow(comment) for comment in all_words]
 
# 设置LDA模型的参数
num_topics = 5  # 假设我们想提取5个主题
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

# 将主题内容存入excel中
topic_list = []
for text_bow in corpus:
    topic_distribution = lda_model[text_bow]
    max_topic = max(topic_distribution, key=lambda x: x[1])[0]
    topic_list.append(f"主题{max_topic + 1}")
data = {'主题名': topic_list, '分词后内容': all_texts}
df = pd.DataFrame(data)
df = df.sort_values(by='主题名')
df.to_excel('data主题分类后.xlsx', index=False)

# 打印每个主题的关键字
for idx, topic in lda_model.print_topics(-1):
    print(f"主题 {idx + 1}: {topic}")
print('\n结束时间：' + str(datetime.datetime.now()))

开始时间：2024-12-24 09:40:14.933968
主题 1: 0.067*"幸福" + 0.036*"希望" + 0.024*"开心" + 0.018*"大笑" + 0.013*"我爱你" + 0.012*"想要" + 0.012*"可怜" + 0.010*"天天开心" + 0.010*"眼泪" + 0.010*"永远"
主题 2: 0.020*"流泪" + 0.020*"世界" + 0.016*"文案" + 0.015*"珍惜" + 0.011*"恭喜" + 0.011*"有人" + 0.010*"可爱" + 0.010*"身边" + 0.009*"真心" + 0.008*"期待"
主题 3: 0.022*"罗生门" + 0.021*"大哭" + 0.015*"难过" + 0.014*"讨厌" + 0.014*"好像" + 0.013*"朋友" + 0.011*"回忆" + 0.011*"不想" + 0.010*"相遇" + 0.009*"遗憾"
主题 4: 0.092*"加油" + 0.056*"喜欢" + 0.036*"冬天" + 0.016*"一年" + 0.013*"谢谢" + 0.013*"选择" + 0.010*"自由" + 0.009*"青春" + 0.008*"真诚" + 0.008*"离开"
主题 5: 0.029*"真的" + 0.027*"喜欢" + 0.018*"生活" + 0.017*"永远" + 0.017*"时间" + 0.016*"记得" + 0.015*"好好" + 0.010*"季节" + 0.008*"回忆" + 0.008*"不到"

结束时间：2024-12-24 09:44:41.459228


## 词共现分析

In [4]:
from collections import defaultdict, Counter
print('开始时间：' + str(datetime.datetime.now()))
# 生成词共现矩阵
co_occurrence_matrix = defaultdict(Counter)
window_size = 2  # 定义窗口大小

# 遍历每个句子
for sentence in all_words:
    # 遍历句子中的每个单词，计算窗口内的共现词
    for i, word in enumerate(sentence):
        for j in range(1, window_size + 1):
            # 检查左侧共现词
            if i - j >= 0:
                left_word = sentence[i - j]
                co_occurrence_matrix[word][left_word] += 1
            # 检查右侧共现词
            if i + j < len(sentence):
                right_word = sentence[i + j]
                co_occurrence_matrix[word][right_word] += 1

# 定义要查询的单词列表（选取词频统计中三首歌同时出现次数最多的四个词）
target_words = ['喜欢', '幸福', '永远', '真的']
# 打印每个目标单词的前10名共现词及其出现次数
for target_word in target_words:
    if target_word in co_occurrence_matrix:
        co_occurrences = co_occurrence_matrix[target_word]
        sorted_co_occurrences = sorted(co_occurrences.items(), key=lambda x: x[1], reverse=True)
        top_20_co_occurrences = sorted_co_occurrences[:10]
        print(f'{target_word} 的前20名共现词及其出现次数:')
        for co_word, count in top_20_co_occurrences:
            print(f'  {co_word}: {count}',end='')
        print()
    else:
        print(f'{target_word} 没有共现词。')

print('\n结束时间：' + str(datetime.datetime.now()))

开始时间：2024-12-24 09:44:41.468079
喜欢 的前20名共现词及其出现次数:
  喜欢: 2812  真的: 907  永远: 307  朋友: 298  感觉: 243  好像: 228  讨厌: 221  大哭: 204  希望: 200  幸福: 198
幸福 的前20名共现词及其出现次数:
  幸福: 1176  痛苦: 410  希望: 389  永远: 384  眼泪: 361  好像: 268  听歌: 231  短暂: 215  喜欢: 198  真的: 183
永远 的前20名共现词及其出现次数:
  永远: 658  幸福: 384  喜欢: 307  希望: 273  记得: 199  分开: 198  身边: 180  回忆: 176  开心: 156  我会: 146
真的 的前20名共现词及其出现次数:
  喜欢: 907  真的: 494  大哭: 268  流泪: 204  幸福: 183  有人: 181  难过: 158  希望: 154  时间: 150  眼泪: 142

结束时间：2024-12-24 09:44:43.789535
